### 1. Setup
We import the tools we need (`sqlite3` and `pandas`) and connect to our database file.

In [19]:
import sqlite3 as sql
import pandas as pd

DB_connection = sql.connect('bibliotheque.db')
cursor = DB_connection.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")
print('the database bibliotheque has been connected seccussefully')

the database bibliotheque has been connected seccussefully


### 2. Available Books
We are looking for all the books that are currently in the library and ready to be borrowed.

In [4]:
#===============================================================
# 1. lister les livres disponibles
#===============================================================
try:
    disponible = 1
    cursor.execute("SELECT livre_id, titre FROM livres WHERE disponible = ?", (disponible,))
    livres_disponibles = cursor.fetchall()
    df_livres_disponibles = pd.DataFrame(livres_disponibles, columns=['livre_id', 'titre'])
    print(f"Livres disponibles :\n \n{df_livres_disponibles}")

except sql.Error as e :
    print(f"Error : {e}")

Livres disponibles :
 
   livre_id                       titre
0         1                    Germinal
1         3         Kafka sur le rivage
2         5                L’Alchimiste
3         7      Le Meilleur des mondes
4         8  Ainsi parlait Zarathoustra
5        10                   Le Procès
6        11                      Carrie
7        12           L’Île mystérieuse


### 3. Borrowed Books
We are making a list of all the books that people have taken home right now.

In [ ]:
#===============================================================
# 2. lister les livres empruntes
#===============================================================

try:
    cursor.execute("""SELECT 
                        k.livre_id,
                        l.titre,
                        k.date_emprunt,
                        k.date_retour_prevue
                        FROM emprunts k JOIN livres l ON k.livre_id = l.livre_id;""")
    livres_empruntes = cursor.fetchall()
    df_livres_empruntes = pd.DataFrame(livres_empruntes,
                columns=['livre_id', 'titre', 'date_emprunt', 'date_retour_prevue'])
    print(f"Livres empruntes :\n\n{df_livres_empruntes}")
except sql.Error as e:
    print(f"Error : {e}")


Livres empruntes :

    livre_id                                titre date_emprunt  \
0          2  Harry Potter à l’école des sorciers   2024-10-01   
1          4                              Shining   2024-10-02   
2          6                 Cent ans de solitude   2024-10-03   
3          9                    Le Nom de la rose   2024-10-04   
4          1                             Germinal   2024-10-05   
5          7               Le Meilleur des mondes   2024-10-06   
6          5                         L’Alchimiste   2024-10-07   
7          3                  Kafka sur le rivage   2024-10-08   
8         10                            Le Procès   2024-10-09   
9          2  Harry Potter à l’école des sorciers   2024-10-10   
10        11                               Carrie   2024-10-11   

   date_retour_prevue  
0          2024-10-08  
1          2024-10-09  
2          2024-10-10  
3          2024-10-11  
4          2024-10-12  
5          2024-10-13  
6          2024-10-

### 4. Authors being read
We want to see which authors have books that are currently borrowed by people.

In [ ]:
#===============================================================
# 3. lister les auteurs ayant emprunté des livres
#===============================================================

try:
    cursor.execute("""SELECT DISTINCT a.auteur_id, a.nom, a.prenom, l.titre FROM emprunts e
                            JOIN livres l ON e.livre_id = l.livre_id
                            JOIN auteurs a ON l.auteur_id = a.auteur_id
                            WHERE e.date_emprunt IS NOT NULL
                            ORDER BY e.date_emprunt ASC;""")
    auteurs_emprunt_livres = cursor.fetchall()
    df_auteurs_emprunt_livres = pd.DataFrame(auteurs_emprunt_livres,
                    columns=['auteur_id', 'nom', 'prenom', 'titre'])

    print(f"Auteurs ayant emprunté des livres :\n\n{df_auteurs_emprunt_livres}")

except sql.Error as e:
    print(f"Error : {e}")


Auteurs ayant emprunté des livres :

   auteur_id       nom   prenom                                titre
0          2   Rowling     J.K.  Harry Potter à l’école des sorciers
1          4      King  Stephen                              Shining
2          6    Garcia  Gabriel                 Cent ans de solitude
3          9       Eco  Umberto                    Le Nom de la rose
4          1      Zola    Émile                             Germinal
5          7    Huxley   Aldous               Le Meilleur des mondes
6          5    Coelho    Paulo                         L’Alchimiste
7          3  Murakami   Haruki                  Kafka sur le rivage
8         10     Kafka    Franz                            Le Procès
9          4      King  Stephen                               Carrie


### 5. Overdue Books
We are checking which books have not been returned yet to the library.

In [20]:
#===============================================================
# 4. lister les livres non retournés
#===============================================================

try:
    cursor.execute("""SELECT l.titre, er.nom, er.prenom, et.date_emprunt, et.date_retour_prevue 
                                    FROM livres l
                                    JOIN emprunts et ON l.livre_id = et.livre_id
                                    JOIN emprunteurs er ON er.emprunteur_id = et.emprunteur_id
                                    WHERE et.date_retour_effective IS NULL
                                    ORDER BY et.date_emprunt ASC;""")
    emprunts_non_retournes = cursor.fetchall()
    df_emprunts_non_retournes = pd.DataFrame(emprunts_non_retournes,
                 columns=['titre', 'nom', 'prenom', 'date_emprunt', 'date_retour_prevue'])

    print(f"Livres :\n\n{df_emprunts_non_retournes}")

except sql.Error as e:
    print(f"Error : {e}")


Livres :

                                 titre        nom   prenom date_emprunt  \
0  Harry Potter à l’école des sorciers     Benali  Yassine   2024-10-01   
1                 Cent ans de solitude     Diallo   Moussa   2024-10-03   
2                    Le Nom de la rose    Lemoine    Sarah   2024-10-04   
3               Le Meilleur des mondes  El Amrani   Khalid   2024-10-06   
4                  Kafka sur le rivage     Dupuis   Claire   2024-10-08   
5                            Le Procès       Khan    Amina   2024-10-09   
6  Harry Potter à l’école des sorciers      Lopez   Carlos   2024-10-10   
7                               Carrie     Benali  Yassine   2024-10-11   

  date_retour_prevue  
0         2024-10-08  
1         2024-10-10  
2         2024-10-11  
3         2024-10-13  
4         2024-10-15  
5         2024-10-16  
6         2024-10-17  
7         2024-10-18  


### 6. Books by Genre
We are counting how many books we have for each category like Roman, Fantasy, or Horror.

In [13]:
#===============================================================
# 5. lister les genres des livres
#===============================================================

try:
    cursor.execute(""" SELECT g.nom_genre, COUNT(l.genre_id) AS total_livres
                            FROM genre g LEFT JOIN livres l 
                            ON g.genre_id = l.genre_id
                            GROUP BY l.genre_id
                            ORDER BY total_livres DESC;""")
    genres = cursor.fetchall()
    df_genres = pd.DataFrame(genres, columns=['nom_genre', 'total_livres'])

    print(f"Genres :\n\n{df_genres}")

except sql.Error as e:
    print(f"Error : {e}")

Genres :

         nom_genre  total_livres
0            Roman             4
1          Horreur             2
2        Classique             2
3         Policier             1
4      Philosophie             1
5          Fantasy             1
6  Science-fiction             1
7            Essai             0


### 7. VIP Borrowers
We are looking for people who have borrowed more than one book to see who our best readers are.

In [15]:
#===============================================================
# 6. lister les emprunts plusieurs livres
#===============================================================

try:
    total_emprunts_condition = 1
    cursor.execute(""" SELECT er.emprunteur_id, er.nom, er.prenom, COUNT(et.emprunteur_id) AS total_emprunts
                                FROM emprunteurs er JOIN emprunts et ON er.emprunteur_id = et.emprunteur_id
                                GROUP BY et.emprunteur_id
                                HAVING total_emprunts > ?;""",(total_emprunts_condition,))
    emprunts_plusieurs_livres = cursor.fetchall()
    df_emprunts_plusieurs_livres = pd.DataFrame(emprunts_plusieurs_livres,
             columns=['emprunteur_id', 'nom', 'prenom', 'total_emprunts'])
             
    print(f"Emprunts plusieurs livres :\n\n {df_emprunts_plusieurs_livres}")

except sql.Error as e:
    print(f"Error : {e}")

Emprunts plusieurs livres :

    emprunteur_id     nom   prenom  total_emprunts
0              1  Benali  Yassine               2


### 8. Most Popular Genre
We are finding out which type of book is the most popular among all our readers.

In [16]:
#===============================================================
# 7. lister le genre le plus emprunté
#===============================================================

try:
    cursor.execute("""SELECT g.nom_genre, COUNT(g.nom_genre) AS frequency_borrowed_genre
                        FROM livres l
                        JOIN genre g ON l.genre_id = g.genre_id
                        JOIN emprunts et ON l.livre_id = et.livre_id
                        GROUP BY g.nom_genre
                        ORDER BY frequency_borrowed_genre DESC;""")

    most_borrowed_genre = cursor.fetchone()
    df_most_borrowed_genre = pd.DataFrame([most_borrowed_genre], 
                columns=['nom_genre', 'frequency_borrowed_genre'])

    print(f"Most borrowed genre :\n\n{df_most_borrowed_genre}")

except sql.Error as e:
    print(f"Error : {e}")

Most borrowed genre :

  nom_genre  frequency_borrowed_genre
0     Roman                         3


In [21]:
if DB_connection:
    DB_connection.commit()
    DB_connection.close()                    
    print("Database connection has been closed safely.")

Database connection has been closed safely.
